In [4]:
import pickle
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from proglearn.forest import UncertaintyForest
import time
from collections import Counter

In [5]:
(X, Y), (X_test_org, Y_test_org) = pickle.load(open('/home/weiwya/cifar_100_Bit_m-r101x1_embd.p', 'rb'))
print(X.shape, X_test_org.shape)

clusters = pickle.load(open('/home/weiwya/cifar_tenth_data_clusters.pkl', 'rb'))


(50000, 2048) (10000, 2048)


In [6]:
def gen_fine_level_data(X, fine_label, coarse_label,  X_test,  fine_label_test, coarse_label_test, wanted_idx):
    selected = np.where(coarse_label==wanted_idx)[0]
    X_selected = X[selected]
    Y_fine_selected = fine_label[selected]
    le = LabelEncoder().fit(Y_fine_selected)
    Y_fine_selected = le.transform(Y_fine_selected)
    
    selected_test = np.where(coarse_label_test == wanted_idx)[0]
    X_test_selected = X_test[selected_test]
    Y_fine_test_selected = fine_label_test[selected_test]
    Y_fine_test_selected = le.transform(Y_fine_test_selected)
    return X_selected, Y_fine_selected, X_test_selected, Y_fine_test_selected, le


def gen_fine_clf (X, Y_coarse, Y_fine, X_test, Y_coarse_test, Y_fine_test, wanted_idx, n_estimators=100, max_depth=10):
    X_fine_train, Y_fine_train, X_fine_test, Y_fine_test, le = gen_fine_level_data(X, Y_fine, Y_coarse, X_test, Y_fine_test, Y_coarse_test, wanted_idx )
    
    print(X_fine_train.shape, X_fine_test.shape)
    clf_fine = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_fine.fit(X_fine_train, Y_fine_train)
    
#     pp = clf_fine.predict(X_fine_test, task_id=0)    
#     print(classification_report( Y_fine_test, pp))
    
    return clf_fine, le

def eval_hieracy_cluster_one_step(data, coarse_clf, fine_clfs):
    corse_level = coarse_clf.predict(data)
    res = []
    for i, c in enumerate(corse_level):
        fine_clf, le = fine_clfs[c]
        p = fine_clf.predict(data[i].reshape(1,-1))
        res.append(le.inverse_transform(p)[0])

    return res

def eval_hieracy_cluster_two_step(data, coarse_clf, fine_clfs):
    #get Corase level probality
    print('done coarse %s ' %(data.shape[0]))
    coarse_prob = coarse_clf.predict_proba(data)
    fine_probs, fine_res = [],[]
    
    for idx, (fine_clf, le) in fine_clfs.items():
        fine_prob = fine_clf.predict_proba(data)
        classes = fine_prob.argmax(axis=1)
        print(classes.shape)
        classes = le.inverse_transform(classes)
        #baysian
        fine_prob = coarse_prob[:, idx].reshape(-1,1) * fine_prob
        
        fine_prob = np.max(fine_prob, axis=1)
        fine_probs.append(fine_prob)
        fine_res.append(classes)
        print('done fine %s' %idx)

        
    fine_probs  = np.array(fine_probs)
    wanted  = fine_probs.argmax(axis=0)
    res = []
    for idx, w in enumerate(wanted):
        res.append(fine_res[w][idx])
    return res

def hieracy_cluster(X_train, Y_train, X_test, Y_test, used_hierarchy, n_estimators=100, max_depth=10 ):
    #relabel classes to hierarchy labels
    hierarchy_relabel = {i:used_hierarchy[i] for i in range(used_hierarchy.shape[0])    }
    
    #change fine label to corse label
    Y_train_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_train])
    Y_test_hierarchy = np.array([hierarchy_relabel[x[0]] for x in Y_test])
    
    clf_coarse = UncertaintyForest(n_estimators=n_estimators,
                        max_depth=max_depth)
    clf_coarse.fit(X_train, Y_train_hierarchy)
    
    print('done train corase labels')
    
    fine_clfs= {}
    for wanted_idx in np.unique(used_hierarchy):
        print('processing %s' %wanted_idx)
        fine_clf, le = gen_fine_clf(X_train, Y_train_hierarchy, Y_train, X_test, Y_test_hierarchy, 
                                    Y_test, wanted_idx, n_estimators=n_estimators, max_depth=max_depth )
        fine_clfs[wanted_idx] = (fine_clf, le)
    
    return clf_coarse, fine_clfs

In [11]:
#just use 1st cluster generate by hayen
generated_cluster = clusters[0]

u, c = np.unique(generated_cluster, return_counts=True)

total = len(generated_cluster)


print(u,c)
n_iter = 10
n_trees = 100
max_depth =10
clfs = []
accuracies_1step = []
accuracies_2step = []

master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_iter)




for iteration in range(n_iter):
    start = time.time()
    seed =  seeds[iteration]
    print(iteration)
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.9, random_state=seed)
    print(X_train.shape, X_test_org.shape)
    
    samples = [-1]*total
    idx = np.arange(total)
    for val, count in zip(u,c):
        if len(idx) == count:
            selected = idx
        else:
            selected = np.random.choice(idx, count, replace=False)
            idx = np.array([i for i in idx if i not in selected])
        for s in selected:
            samples[s] = val
    random_clusters = np.array(samples)
        
    clf_coarse, clf_fines = hieracy_cluster(X_train, Y_train, X_test_org, Y_test_org, random_clusters, n_estimators=n_trees, 
                                            max_depth=max_depth)
    
    
    
    clfs.append((clf_coarse, clf_fines))
    pp = eval_hieracy_cluster_one_step(X_test_org, clf_coarse, clf_fines)
    
    accuracies_1step.append(accuracy_score(Y_test_org, pp))
    
    pp = eval_hieracy_cluster_two_step(X_test_org, clf_coarse, clf_fines)
    accuracies_2step.append(accuracy_score(Y_test_org, pp))

    
    print(accuracies_1step[-1], accuracies_2step[-1])
    print('took %s' %(time.time() - start))
    print()
    

with open('/home/weiwya/cirar_random_10_10_run.p', 'wb') as handle:
    pickle.dump((clfs, accuracies_1step, accuracies_2step), handle)

print(np.average(accuracies_1step), np.average(accuracies_2step))
 
    
        

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26] [ 4  5  5 12  4  6  1 12  6  1  4  6  1  2  1  1  2  5  1  4  1  2  3  5
  3  2  1]
0
(5000, 2048) (10000, 2048)
done train corase labels
processing 0
(227, 2048) (400, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 1
(276, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 2
(260, 2048) (500, 2048)


...

processing 25
(105, 2048) (200, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 26
(49, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


done coarse 10000 
(10000,)
done fine 0
(10000,)
done fine 1
(10000,)
done fine 2
(10000,)
done fine 3
(10000,)
done fine 4
(10000,)
done fine 5
(10000,)
done fine 6
(10000,)
done fine 7
(10000,)
done fine 8
(10000,)
done fine 9
(10000,)
done fine 10
(10000,)
done fine 11
(10000,)
done fine 12
(10000,)
done fine 13
(10000,)
done fine 14
(10000,)
done fine 15
(10000,)
done fine 16
(10000,)
done fine 17
(10000,)
done fine 18
(10000,)
done fine 19
(10000,)
done fine 20
(10000,)
done fine 21
(10000,)
done fine 22
(10000,)
done fine 23
(10000,)
done fine 24
(10000,)
done fine 25
(10000,)
done fine 26
0.3068 0.422
took 1269.3341507911682

0.29508 0.39742000000000005


In [12]:
#just use 1st cluster generate by hayen
condition_cluster = pickle.load(open('/home/weiwya/cifar_cond_means_clusters.pkl', 'rb'))

generated_cluster = condition_cluster[0]

u, c = np.unique(generated_cluster, return_counts=True)

total = len(generated_cluster)


print(u,c)
n_iter = 10
n_trees = 100
max_depth =10
clfs_cond = []
accuracies_1step_cond = []
accuracies_2step_cond = []

master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_iter)




for iteration in range(n_iter):
    start = time.time()
    seed =  seeds[iteration]
    print(iteration)
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.9, random_state=seed)
    print(X_train.shape, X_test_org.shape)
    
    samples = [-1]*total
    idx = np.arange(total)
    for val, count in zip(u,c):
        if len(idx) == count:
            selected = idx
        else:
            selected = np.random.choice(idx, count, replace=False)
            idx = np.array([i for i in idx if i not in selected])
        for s in selected:
            samples[s] = val
    random_clusters = np.array(samples)
        
    clf_coarse, clf_fines = hieracy_cluster(X_train, Y_train, X_test_org, Y_test_org, random_clusters, n_estimators=n_trees, 
                                            max_depth=max_depth)
    
    
    
    clfs_cond.append((clf_coarse, clf_fines))
    pp = eval_hieracy_cluster_one_step(X_test_org, clf_coarse, clf_fines)
    
    accuracies_1step_cond.append(accuracy_score(Y_test_org, pp))
    
    pp = eval_hieracy_cluster_two_step(X_test_org, clf_coarse, clf_fines)
    accuracies_2step_cond.append(accuracy_score(Y_test_org, pp))

    
    print(accuracies_1step_cond[-1], accuracies_2step_cond[-1])
    print('took %s' %(time.time() - start))
    print()
    

with open('/home/weiwya/cirar_random_conditional_10_10_run.p', 'wb') as handle:
    pickle.dump((clfs_cond, accuracies_1step_cond, accuracies_2step_cond), handle)

print(np.average(accuracies_1step_cond), np.average(accuracies_2step_cond))
 

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29] [ 5  2 10  7  4  1  2  5  4  5  6  1  3  2  2  1  6  3  4  3  4  2  5  2
  2  5  1  1  1  1]
0
(5000, 2048) (10000, 2048)
done train corase labels
processing 0
(278, 2048) (500, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 1
(117, 2048) (200, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 2
(489, 2048) (1000, 2048)


...

processing 26
(46, 2048) (100, 2048)
processing 27
(48, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


processing 28
(49, 2048) (100, 2048)
processing 29
(53, 2048) (100, 2048)


/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/weiwya/screen_understanding/screen_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


done coarse 10000 
(10000,)
done fine 0
(10000,)
done fine 1
(10000,)
done fine 2
(10000,)
done fine 3
(10000,)
done fine 4
(10000,)
done fine 5
(10000,)
done fine 6
(10000,)
done fine 7
(10000,)
done fine 8
(10000,)
done fine 9
(10000,)
done fine 10
(10000,)
done fine 11
(10000,)
done fine 12
(10000,)
done fine 13
(10000,)
done fine 14
(10000,)
done fine 15
(10000,)
done fine 16
(10000,)
done fine 17
(10000,)
done fine 18
(10000,)
done fine 19
(10000,)
done fine 20
(10000,)
done fine 21
(10000,)
done fine 22
(10000,)
done fine 23
(10000,)
done fine 24
(10000,)
done fine 25
(10000,)
done fine 26
(10000,)
done fine 27
(10000,)
done fine 28
(10000,)
done fine 29
0.3369 0.4637
took 1242.747391462326

0.29066000000000003 0.40312000000000003


In [13]:
print("random average:%s std_err: %s " %(np.average(accuracies_2step), np.std(accuracies_2step)/np.sqrt(n_iter)   ))
print("random_conditional average:%s std_err: %s " %(np.average(accuracies_2step_cond), np.std(accuracies_2step_cond)/np.sqrt(n_iter)   ))

random average:0.39742000000000005 std_err: 0.007412851003493863 
random_conditional average:0.40312000000000003 std_err: 0.00989053891352741 
